In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorAssembler

from datetime import datetime

In [ ]:
#pyspark init
builder = SparkSession.Builder().appName('taxi_duration_highAPI')
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/03/28 21:59:53 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/28 21:59:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/28 21:59:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read input file (train.csv only, since the test.csv does not have label for each data point)
raw_data = spark.read.csv('train.csv', header=True, inferSchema=True)

In [4]:
#Cast the pickup and dropoff string values into timestamps.
casted_data = raw_data.withColumns({
                        'pickup_datetime' : raw_data['pickup_datetime'].cast('timestamp'),
                        'dropoff_datetime' : raw_data['dropoff_datetime'].cast('timestamp')
                    })

In [5]:
#Get usable columns from the dataframe
#Also convert pickup and dropoff timestamps into time elements and encode `store_and_fwd_flag` feature into binary values
extracted_df = casted_data.selectExpr(
                'YEAR(pickup_datetime)    AS pickup_year',
                'MONTH(pickup_datetime)   AS pickup_month',
                'DAY(pickup_datetime)     AS pickup_day',
                'HOUR(pickup_datetime)    AS pickup_hour',
                'MINUTE(pickup_datetime)  AS pickup_min',
                'SECOND(pickup_datetime)  AS pickup_sec',
                'YEAR(dropoff_datetime)   AS dropoff_year',
                'MONTH(dropoff_datetime)  AS dropoff_month',
                'DAY(dropoff_datetime)    AS dropoff_day',
                'HOUR(dropoff_datetime)   AS dropoff_hour',
                'MINUTE(dropoff_datetime) AS dropoff_min',
                'SECOND(dropoff_datetime) AS dropoff_sec',
                'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'CASE WHEN store_and_fwd_flag == "Y" THEN 1 ELSE 0 END AS store_and_fwd_flag',
                'trip_duration'
            )

In [6]:
#Extract features name
cols = [col for col in extracted_df.columns if col != 'trip_duration']

#Use VectorAssembler to transform feature columns into a single vector
assember = VectorAssembler(inputCols=cols, outputCol='features')
data = assember.transform(extracted_df).select('features', 'trip_duration')

In [7]:
#Split data into training and testing sets

###training set proportion parameter:
train_size = 0.8
###

train, test = data.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

In [ ]:
estimator = DecisionTreeRegressor(featuresCol='features', labelCol='trip_duration')
model = estimator.fit(train)

25/03/28 22:02:19 WARN MemoryStore: Not enough space to cache rdd_25_0 in memory! (computed 18.5 MiB so far)
25/03/28 22:02:19 WARN BlockManager: Persisting block rdd_25_0 to disk instead.
25/03/28 22:02:19 WARN MemoryStore: Not enough space to cache rdd_25_1 in memory! (computed 18.5 MiB so far)
25/03/28 22:02:19 WARN BlockManager: Persisting block rdd_25_1 to disk instead.


In [17]:
train.show(1)

25/03/28 22:06:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-------------+
|            features|trip_duration|
+--------------------+-------------+
|[2016.0,1.0,1.0,0...|          849|
+--------------------+-------------+
only showing top 1 row



In [16]:
model.featureImportances

SparseVector(18, {2: 0.9798, 3: 0.0033, 7: 0.0011, 8: 0.0005, 13: 0.0069, 14: 0.0014, 15: 0.0051, 16: 0.0019})